In [1]:
import pandas as pd
import numpy as np



In [2]:
#pip install pyspellchecker

In [3]:
# data loading
path = '/Users/maida/Downloads/'
data = pd.read_json(path + 'data.json')
data.head()

,source,text,created_at,retweet_count,id_str
0,Twitter for iPhone,White House news conference at 5:00 P.M. Easte...,2020-03-30 20:50:35+00:00,14441,1244728753617620992
1,Twitter for iPhone,https://t.co/2hKJkP5Z6N,2020-03-30 17:46:15+00:00,15520,1244682364284014592
2,Twitter for iPhone,"On #NationalDoctorsDay, we recognize the remar...",2020-03-30 17:11:59+00:00,19753,1244673740866191360
3,Twitter for iPhone,https://t.co/nzWJ8ViwbZ,2020-03-30 17:05:33+00:00,39114,1244672122414338048
4,Twitter for iPhone,Nancy Pelosi and the Democrats delayed the Wor...,2020-03-30 11:17:10+00:00,43360,1244584449309892608


In [ ]:
import string
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import words

"""
This is preprocessing section. Following steps were taken:

    1. punctuation removal with '!' exception
    2. numbers removal
    3. url removals
    4. normalization to lowercase with exception for isupper()
    5. string to tokens
    6. stopwords removal
    7. stemming
    8. normalization of elongated words
"""

stop_words = list(stopwords.words('english'))

data.dtypes

# changing float64 to string
data['text'] = data['text'].astype(str)

# remove punctuation
def remove_punctuations(text):
    for punctuation in string.punctuation:
        if punctuation != '!': # leave exclamation mark
            text = text.replace(punctuation, '')
    return text

# remove numbers 
def remove_numbers(text):
    return re.sub('[0-9]+', '', text)

# remove urls
def remove_urls(text):
    return re.sub(r"http\S+", "", text)

# if word starts with uppercase --> lowercase, if all chars are uppercase --> do nothing
def lower_case(tokens):
    tokens = [(w.lower() if not w.isupper() else w) for w in tokens]
    return tokens
            
# print(lower_case(['AAAAA', 'army', 'Army'])) # test function
# print(remove_urls(remove_punctuations(remove_numbers('740 test test 3 99ma http://cnn.com')))) # test function

#remove elongated words
def remove_elongated(text):
    el = []
    setofwords = set(words.words())
    for word in text.split():
        if word in setofwords:
            pass
        else:
            word=re.sub(r'(?i)(.)\1+', r'\1', word)
        el.append(word)
    return el

#test = 'Aweeeesome president Trump greeeeat good Ameerica '
#print(remove_elongated(test))

data['text'] = data.apply(lambda x: remove_numbers(x['text']), axis=1)
data['text'] = data.apply(lambda x: remove_punctuations(x['text']), axis=1)
data['text'] = data.apply(lambda x: remove_urls(x['text']), axis=1)
data['text'] = data.apply(lambda x: remove_elongated(x['text']), axis=1)
# tokenize
data['tokens'] = data.apply(lambda x: word_tokenize(x['text']), axis=1)

# lower_case
data['tokens'] = data.apply(lambda x: lower_case(x['tokens']), axis=1)
            
# remove stop words
data['tokens'] = data.apply(lambda x: [w for w in x['tokens'] if w not in stop_words], axis=1)

# stemming
ps = PorterStemmer() 
data['tokens'] = data.apply(lambda x: [ps.stem(w) for w in x['tokens']], axis=1)

# enlogated words, eg. 'aweeesome'; essentially we are searching for words that contain 3+ same [a-z] characters in a row


In [5]:
def check_elongated(text):
    el = []
    regex_el = r"\w*(\w)\1\w*"
    for word in text.split():
        el.append(re.search(regex_el, word))
    return el
    
test = 'Aweeeesome president Trump greeeeat Ameerica good book'
print(check_elongated(test))

[<re.Match object; span=(0, 10), match='Aweeeesome'>, None, None, <re.Match object; span=(0, 8), match='greeeeat'>, <re.Match object; span=(0, 8), match='Ameerica'>, <re.Match object; span=(0, 4), match='good'>, <re.Match object; span=(0, 4), match='book'>]


['Awesome', 'president', 'Trump', 'great', 'good', 'America']
True


('book', 0.29832935560859186)